# Encryption in CKKS

## Introduction

In [1]:
text = """Now the have seen why we work on polynomials of $\mathbb{Z}_q[X]/(X^N+1)$, and how we could get an encryption scheme based on this, let's see how we can define addition and multiplication ciphertexts to have an homomorphic encryption scheme.

So we said that we have a secret $s$, and a public key $p = (b,a) = (-a.s +e, a)$. To encrypt a message $\mu$ we simply output $c = (\mu + b, a)$, and to decrypt it with $s$ we evaluate $c_0 + c_1.s$ which will approximately gives the original message.

Now suppose we have two messages, $\mu$ and $\mu'$ and that we encrypt them into $c = (c_0,c_1)$ and $c' = (c_0',c_1')$. Then $c_{add} = c + c' = (c_0 + c_0',c_1 + c_1')$ is a correct encryption of $\mu + \mu'$, i.e. when we decrypt it using $s$ we get (approximatively) $\mu + \mu'$.

Indeed, the decryption mechanism of $c_{add}$ yields $c_{add,0} + c_{add,1}.s = c_0 + c_0' + (c_1 + c_1').s = c_0 + c_1.s + c_0' + c_1'.s = \mu + \mu' + 2e \approx \mu + \mu'$ because we said that $e$ is negligible.

What this means is that if you add ciphertexts, and you decrypt them, you will get the addition of the plaintexts! This  means that with this simple scheme, you can allow someone to perform additions on encrypted data, and the user can still decrypt it and get the correct result. This is our first step toward an homomorphic encryption scheme.

Nonetheless, we still need to define the multiplication on ciphertexts, which is more involved. Indeed, our goal will be to find a ciphertext $c_{mult}$ such that when we decrypt it with the secret key $s$, we get the product of the plaintexts. To do this, one way of seeing this problem is noticing that the decryption can be seen as a polynomial in the secret key $s$. Indeed it is 

"""

In [2]:
import re

mystring = 'This string $aaz$ has a $qsfdfsqd$ in it'

pattern = re.compile(r'(\$)(.*?)(\$)')

newstring = pattern.sub(r"\(\2\)", text)
newstring

"Now the have seen why we work on polynomials of \\(\\mathbb{Z}_q[X]/(X^N+1)\\), and how we could get an encryption scheme based on this, let's see how we can define addition and multiplication ciphertexts to have an homomorphic encryption scheme.\n\nSo we said that we have a secret \\(s\\), and a public key \\(p = (b,a) = (-a.s +e, a)\\). To encrypt a message \\(\\mu\\) we simply output \\(c = (\\mu + b, a)\\), and to decrypt it with \\(s\\) we evaluate \\(c_0 + c_1.s\\) which will approximately gives the original message.\n\nNow suppose we have two messages, \\(\\mu\\) and \\(\\mu'\\) and that we encrypt them into \\(c = (c_0,c_1)\\) and \\(c' = (c_0',c_1')\\). Then \\(c_{add} = c + c' = (c_0 + c_0',c_1 + c_1')\\) is a correct encryption of \\(\\mu + \\mu'\\), i.e. when we decrypt it using \\(s\\) we get (approximatively) \\(\\mu + \\mu'\\).\n\nIndeed, the decryption mechanism of \\(c_{add}\\) yields \\(c_{add,0} + c_{add,1}.s = c_0 + c_0' + (c_1 + c_1').s = c_0 + c_1.s + c_0' + c_1'

Now that we saw how vectors can be encoded into polynomials in the CKKS scheme, we will see how one can perform encryption and decryption. We will see the high picture view of encryption and decryption, and it will actually be easier than encoding and decoding. 

CKKS is a public key encryption scheme, where a secret key and a public key are generated. While the public key is used for encryption and can be shared, the private key is used for decryption and must be kept secret.

The foundation of CKKS, and many other homomorphic encryption schemes, is the Learning With Error (LWE) problem. The LWE problem is to distinguish noisy pairs of the form $(a_i, b_i) = (a_i, <a_i,s> + e_i)$ from truly random ones in $\mathbb{Z}_q^n \times \mathbb{Z}_q$. Here $a_i, s \in \mathbb{Z}_q^n$, $a_i$ is uniformly sampled, $s$ is our secret, and the $e_i \in \mathbb{Z}_q$ are small noises used to make the problem harder.

The LWE problem is known to be as hard as worst case lattice problems, which are currently secure against attacks from quantum computers. Therefore, we can exploit the fact that finding a secret $s$ from pairs of $(a_i, <a_i,s> + e_i)$ is hard, and build a cryptosystem upon this. 

Suppose we have generated a secret key $s \in \mathbb{Z}_q^n$, and display $n$ pairs of the type $(a_i, <a_i,s> + e_i)$, so we can write this in matrix form $(A, A.s + e)$ with $A \in \mathbb{Z}_q^{n \times n}, e \in \mathbb{Z}_q^n$. As we said, we can consider $p = (-A.s + e, A)$ to be our public key, as the LWE problem states that it is hard to recover the secret key from it, therefore it can be known to anyone. We chose to store $-A.s$ instead of $A.s$ for convenience as we will see later, but it doesn't change the problem.

Then to encrypt and decrypt a message $\mu \in \mathbb{Z}_q^n$ using the public and secret key, we can use the following scheme:
- Encryption of $\mu$ using $p$: output $c = (\mu,0) + p) = (\mu -A.s + e, A) = (c_0, c_1)$.
- Decryption of $c$ using $s$: output $\tilde{\mu} = c_0 + c_1.s = \mu -A.s + e + A.s = \mu + e \approx \mu$

So for the encryption phase, we take the public key, and use it to mask our message $\mu$. The message is thus hidden in the first coordinate of the ciphertext with a the mask $-A.s$. Remember that $A$ is sampled uniformly, so it will indeed mask $\mu$ effectively. To remove it, we can use the second coordinate, which only stores $A$ and combine it with the secret key $s$ to obtain the decryption which is $\mu + e$. Notice here that we do not exactly get the original message $\mu$ but $\mu$ plus some noise $e$. If $e$ is small enough, then the decryption will be close to the original $\mu$.

So we have seen here how to use the LWE problem to build a public crypto scheme that is secure against quantum attacks. The problem with the implementation above, is that while the secret key has size $\mathcal{O}(n)$, the public key has size $\mathcal{O}(n^2)$ due to the matrix $A$ and computations will also require $\mathcal{O}(n^2)$ operations. Because $n$ will dictate the security of our scheme, if we use LWE to construct our scheme, it will be too inefficient in practice as the $\mathcal{O}(n^2)$ size of keys and complexity will make it too impractical.

That is why we will consider the Ring Learning With Error problem, which is a variant of LWE but on rings. Instead of working with vectors in $\mathbb{Z}_q^n$, we will work on polynomials in $\mathbb{Z}_q[X]/(X^N+1)$ (we suppose here that $n$ is a power of 2). So now we draw $a$, $s$, and $e$ from $\mathbb{Z}_q[X]/(X^N+1)$, where $a$ is still sampled uniformly, $s$ is a small secret polynomial, and $e$ is a small noisy polynomial. Switching to RLWE has two main advantadges:
- The key size is no longer quadratic but linear, as we now output the public key $p = (-a.s +e, a)$, where $a.s$ denotes the polynomial product of $a$ with $s$. Because all operations are done between polynomials, both the private and public keys are of size $\mathcal{O}(n)$. 
- Multiplications are done on polynomials, therefore it can be done with a complexity of $\mathcal{O}(n log(n))$ using Discrete Fourier Transform, instead of $\mathcal{O}(n^2)$ because we had to do matrix vector multiplication. 

Therefore by using RLWE instead of LWE, we will much smaller keys, and operations will be even faster, so the preceding scheme becomes much more practical. Moreover, the RLWE is still a hard problem and provides strong security guarantees. Indeed the paper [On Ideal Lattices and Learning with Errors Over Rings](https://eprint.iacr.org/2012/230.pdf) proves that the RLWE problem is as hard as worst-case problems on ideal lattices. 

We know understand why it was important to work with polynomials because they provide the basis for an efficient and secure scheme. Therefore you can understand now why we had to go through all the problems of converting our vectors into polynomials of $\mathbb{Z}_q[X]/(X^N+1)$ and vice-versa.

Now the have seen why we work on polynomials of $\mathbb{Z}_q[X]/(X^N+1)$, and how we could get an encryption scheme based on this, let's see how we can define addition and multiplication ciphertexts to have an homomorphic encryption scheme.

So we said that we have a secret $s$, and a public key $p = (b,a) = (-a.s +e, a)$. To encrypt a message $\mu$ we simply output $c = (\mu + b, a)$, and to decrypt it with $s$ we evaluate $c_0 + c_1.s$ which will approximately gives the original message.

Now suppose we have two messages, $\mu$ and $\mu'$ and that we encrypt them into $c = (c_0,c_1)$ and $c' = (c_0',c_1')$. Then $c_{add} = c + c' = (c_0 + c_0',c_1 + c_1')$ is a correct encryption of $\mu + \mu'$, i.e. when we decrypt it using $s$ we get (approximatively) $\mu + \mu'$.

Indeed, the decryption mechanism of $c_{add}$ yields $c_{add,0} + c_{add,1}.s = c_0 + c_0' + (c_1 + c_1').s = c_0 + c_1.s + c_0' + c_1'.s = \mu + \mu' + 2e \approx \mu + \mu'$ because we said that $e$ is negligible.

What this means is that if you add ciphertexts, and you decrypt them, you will get the addition of the plaintexts! This  means that with this simple scheme, you can allow someone to perform additions on encrypted data, and the user can still decrypt it and get the correct result. This is our first step toward an homomorphic encryption scheme.

Nonetheless, we still need to define the multiplication on ciphertexts, which is more involved. Indeed, our goal will be to find a ciphertext $c_{mult}$ such that when we decrypt it with the secret key $s$, we get the product of the plaintexts. To do this, one way of seeing this problem is noticing that the decryption can be seen as a polynomial in the secret key $s$. Indeed it is 



## Preliminaries

![discrete gaussian](https://raw.githubusercontent.com/dhuynh95/homomorphic_encryption_intro/master/images/discrete_gaussian.PNG)
<center>Discrete Gaussian on lattices of $\mathbb{Z}^2$ (source : <a href="https://eprint.iacr.org/2012/230.pdf">On Ideal Lattices and Learning with Errors Over Rings</a>)</center>

Let's see now in practice how it is implemented. We will use the notation of the [CKKS](https://eprint.iacr.org/2016/421.pdf) paper, and denote for any real $\sigma > 0$, $\mathcal{DG}(\sigma^2)$ the discrete Gaussian distribution, which will be used for the sampling of "small" polynomials. While sampling from a Gaussian is simple in $\mathbb{R}^n$, we will need to define a sampling method on $\mathcal{R}$. Taking advantage of the nice structure of the cyclotomic ring $\mathcal{R}$, CKKS provides the following method to sample from $\mathcal{R}$:

- First sample $z \sim \mathcal{N}(0, \sigma^2 I)$, where $I$ is the identity matrix of dimension $N$.
- Project it on $\mathbb{H} = \{z \in \mathbb{C}^{N} : z_j = \overline{z_{-j}} \}$ using the matrix $U$ defined before for encoding.
- Encode it into a polynomial in $\mathcal{R}$ using the same process for encoding by rounding it in $\sigma(\mathcal{R})$ and encode it with $\sigma^{-1}$.

In addition to $\mathcal{DG}(\sigma^2)$, we will need the following distributions :
- $\mathcal{HWT}(h)$, for $h \in [0, \dots, N]$, which outputs a vector in $\{0,\pm 1\}^N$, where $h$ coordinates are non zero.
- $\mathcal{ZO}(p)$ with $p \in [0,1]$, which outputs a vector in $\{0,\pm 1\}^N$, with a probability $p/2$ to output $\pm 1$, and $1 - p$ to output $0$.

Note that these distributions output vectors, and we will use the generated coefficients to define polynomials whose coefficients are the coordinates of the vector.

Now that we have all the distributions needed for CKKS, we can create the secret keys and public keys.

## Key generation

Now that we have all the distributions needed for CKKS, we can create the secret keys and public keys. To do so, first sample $s \sim \mathcal{HWT}(h)$, then sample $a$ uniformly from $\mathcal{R}_q$ and $e \sim \mathcal{DG}(\sigma^2)$. Let $pk = (-a s + e, a) = (pk_0, pk_1)$. Then $sk$ will be our secret key, and $pk$ our public key.

## Encryption and decryption

To encrypt a plaintext polynomial $m \in \mathcal{R}$, we simply need to sample $e_0, e_1 \sim \mathcal{DG}(\sigma^2)$ and $v \sim \mathcal{ZO}(0.5)$. Then output $c = v . pk + (m + e_0, e_1) = (- a s v + e v + m + e_0, a v + e_1)$. The intuition behind this, is that $e$, $e_0$, $e_1$ are just small noisy polynomials used to slightly disturb the message. However the $- a s v$ polynomial will have much bigger coefficients, as $a$ is chosen uniformly on $\mathcal{R}_q$. Therefore this term will mask $m$ from the rest. 

To decrypt a ciphertext $c = (c_0, c_1)$ we will need to use the secret key $sk$. Decryption of a $c$ is simply done by computing $\tilde{m} = c_0 + c_1 sk = m + e v + e_0 + s e_1$, with $\tilde{m}$ the decrypted polynomial. Because the error polynomials have small coefficients compared to $m$, which was scaled previously by $\Delta$, we have that $\tilde{m} \approx m$. 

![Encryption and decryption](https://raw.githubusercontent.com/dhuynh95/homomorphic_encryption_intro/master/images/encryption_decryption.PNG)
<center>Encryption and decryption (source : Introduction to CKKS, from the workshop "Lattices: From Theory to Practice" 2020).</center>

The figure aboves provides an illustration of this principle. We can see that a plaintext polynomial is encrypted into a couple of polynomials, where we add a mask (the $- a s v$ term) which covers the Most Significant Bits (MSB). This means that the message lies in the Least Significant Bits (LSB), and is hidden by a mask whose coefficients are much larger than the original message. Decrypting consists of removing this mask using the second polynomial and the secret key, therefore recovering the original plaintext.

## Addition

Addition of two ciphertexts is simple. Suppose we have two plaintexts $m_1$ and $m_2$ encoded into $c_1 = (b_1, a_1)$ and $c_2 = (b_2, a_2)$, and that their decryption yield $b_1 + a_1 sk = m_1 + e_1$ and $b_2 + a_2 sk = m_2 + e_2$, with $e_1$ and $e_2$ small error polynomials. Then by outputing $\tilde{c} = (b_1 + b_2, a_1 + a_2)$, we have that the decryption operation will yield $m_1 + m_2 + e$ with $e$ a small error.

Indeed, if we perform decryption $\tilde{c}$ we obtain : $c_0 + c_1 sk = a_0 + b_0 + a_1 sk + b_1 s = m_1 + m_2 + e_1 + e_2 = m_1 + m_2 + e$, with $e = e_1 + e_2$. To see that $e$ is a "small" polynomial, \cite{song} provides more details on the boundaries of $e$. Therefore, we have that adding the polynomial of each ciphertext coordinates yields a correct encryption of the sum of the messages. Thus we define $\texttt{Add}(c_1,c_2) = c_1 + c_2$.

Here is the code we created for encoding and decoding in CKKS.

In [1]:
from numpy.polynomial import Polynomial
import numpy as np

def round_coordinates(coordinates):
    """Gives the integral rest."""
    coordinates = coordinates - np.floor(coordinates)
    return coordinates

def coordinate_wise_random_rounding(coordinates):
    """Rounds coordinates randonmly."""
    r = round_coordinates(coordinates)
    f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)
    
    rounded_coordinates = coordinates - f
    rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
    return rounded_coordinates

class CKKSEncoder:
    """Basic CKKS encoder to encode complex vectors into polynomials."""
    
    def __init__(self, context, scale:float):
        """Initializes with scale."""
        
        M = context.M
        self.context = context
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M
        self.create_sigma_R_basis()
        self.scale = scale
        
        self.slot_count = M // 4
        
    @staticmethod
    def vandermonde(xi: np.complex128, M: int) -> np.array:
        """Computes the Vandermonde matrix from a m-th root of unity."""
        
        N = M //2
        matrix = []
        # We will generate each row of the matrix
        for i in range(N):
            # For each row we select a different root
            root = xi ** (2 * i + 1)
            row = []

            # Then we store its powers
            for j in range(N):
                row.append(root ** j)
            matrix.append(row)
        return matrix
    
    def sigma_inverse(self, b: np.array) -> Polynomial:
        """Encodes the vector b in a polynomial using an M-th root of unity."""

        # First we create the Vandermonde matrix
        A = CKKSEncoder.vandermonde(self.xi, M)

        # Then we solve the system
        coeffs = np.linalg.solve(A, b)

        # Finally we output the polynomial
        p = Polynomial(coeffs)
        return p

    def sigma(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by applying it to the M-th roots of unity."""

        outputs = []
        N = self.M //2

        # We simply apply the polynomial on the roots
        for i in range(N):
            root = self.xi ** (2 * i + 1)
            output = p(root)
            outputs.append(output)
        return np.array(outputs)
    

    def pi(self, z: np.array) -> np.array:
        """Projects a vector of H into C^{N/2}."""

        N = self.M // 4
        return z[:N]


    def pi_inverse(self, z: np.array) -> np.array:
        """Expands a vector of C^{N/2} by expanding it with its
        complex conjugate."""

        z_conjugate = z[::-1]
        z_conjugate = [np.conjugate(x) for x in z_conjugate]
        return np.concatenate([z, z_conjugate])
    
    def create_sigma_R_basis(self):
        """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""

        self.sigma_R_basis = np.array(self.vandermonde(self.xi, self.M)).T

    def compute_basis_coordinates(self, z):
        """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
        output = np.array([np.real(np.vdot(z, b) / np.vdot(b,b)) for b in self.sigma_R_basis])
        return output

    def sigma_R_discretization(self, z):
        """Projects a vector on the lattice using coordinate wise random rounding."""
        coordinates = self.compute_basis_coordinates(z)

        rounded_coordinates = coordinate_wise_random_rounding(coordinates)
        y = np.matmul(self.sigma_R_basis.T, rounded_coordinates)
        return y


    def encode(self, z: np.array) -> Polynomial:
        """Encodes a vector by expanding it first to H,
        scale it, project it on the lattice of sigma(R), and performs
        sigma inverse.
        """
        pi_z = self.pi_inverse(z)
        scaled_pi_z = self.scale * pi_z
        rounded_scale_pi_zi = self.sigma_R_discretization(scaled_pi_z)
        p = self.sigma_inverse(rounded_scale_pi_zi)

        # We round it afterwards due to numerical imprecision
        coef = np.round(np.real(p.coef)).astype(int)
        p = self.context.QPolynomial(coef)
        
        return p


    def decode(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by removing the scale, 
        evaluating on the roots, and project it on C^(N/2)"""
        rescaled_p = p / self.scale
        z = self.sigma(rescaled_p)
        pi_z = self.pi(z)
        return pi_z
        
    def decode_float(self, p:Polynomial) -> np.array:
        pi_z = self.decode(p)
        pi_z = np.real(pi_z)
        return pi_z

To be able to implement CKKS encryption and decryption, we must first generate the secret and public keys. To do so, we will define a few helper functions for arithmetic on $\mathbb{Z}_q[X]/(X^N + 1)$.

In [2]:
from numpy.polynomial import Polynomial

def mod_q(coeffs: np.ndarray, q: int) -> np.ndarray:
    """Reduce modulo q to (-q/2,q/2]"""
    
    r = coeffs % q
    # Coefficients larger than q/2 are sent to (-q/2,0]
    to_cycle = r > (q/2)
    r[to_cycle] = r[to_cycle] - q
    return r

def mod_q_polynomial(p: Polynomial, q: int) -> Polynomial:
    """Reduces modulo q the coefficients of a polynomial"""
    coeffs = p.coef
    coeffs_mod_q = mod_q(coeffs, q)
    p = Polynomial(coeffs_mod_q)
    return p

While it is possible to simply create functions `add(p1,p2)`, and `multiply(p1,p2)` whenever we want to add or multiply polynomials in $\mathbb{Z}_q[X]/(X^N + 1)$, a more heavy but elegant way to do this, is to create custom polynomial objects, which will wrap the `Polynomial` class of Numpy and overload addition and multiplication.

To do this, we create two classes `QPolynomialGenerator`, which will be used to create `QPolynomial` objects, which can be added or multiplied together.

In [3]:
from numpy.polynomial import polynomial as poly
from numpy.polynomial import Polynomial
from __future__ import annotations
        
class QPolynomialGenerator:
    """Polynomial generator for polynomials in Z_q[X]/(X^N + 1)"""
    
    def __init__(self, N: int, q: int):
        self.q = q
        
        coeffs = np.zeros(N + 1)
        coeffs[0] = 1
        coeffs[-1] = 1
        
        self.poly_modulus = Polynomial(coeffs)
    
    def __call__(self, coef) -> QPolynomial:
        """Creates a polynomial from coefficients."""
        return QPolynomial(coef, self.q, self.poly_modulus, self)
    
class QPolynomial:
    """Polynomial in Z_q[X]/(X^N + 1). 

    Addition, substraction, multiplication and division are overloaded with
    the correct operations inZ_q[X]/(X^N + 1)."""

    def __init__(self, coef, q, poly_modulus, generator):
        self.p = Polynomial(coef)
        self.q = q
        self.poly_modulus = poly_modulus
        self.generator = generator

    def __getattr__(self,k):
        """Overload attribute getter to fetch the wrapped polynomial attributes if it
        is not present in the QPolynomial."""
        if k in self.__dict__.keys():
            return getattr(self, k)
        else:
            try:
                return getattr(self.p, k)
            except AttributeError as e:
                pass

    def __mul__(self, right) -> QPolynomial:
        if isinstance(right, QPolynomial):
            right = right.p
        p_mul = self.p * right % self.poly_modulus
        p_mul = self.generator(p_mul.coef)
        return p_mul

    def __rmul__(self, left) -> QPolynomial:
        return self.__mul__(left)

    def __add__(self, right) -> QPolynomial:
        if isinstance(right, QPolynomial):
            right = right.p
        p_add = (self.p + right) % self.poly_modulus
        p_add = self.generator(p_add.coef)
        return p_add

    def __radd__(self, left) -> QPolynomial:
        return self.__add__(left)

    def __sub__(self, qpoly: QPolynomial) -> QPolynomial:
        p_sub = (self.p - qpoly.p) % self.poly_modulus
        p_sub = self.generator(p_sub.coef)
        return p_sub

    def __rsub__(self, qpoly: QPolynomial) -> QPolynomial:
        p_sub = (qpoly.p - self.p) % self.poly_modulus
        p_sub = self.generator(p_sub.coef)
        return p_sub

    def __neg__(self):
        p_neg = - self.p %self.poly_modulus
        p_neg = self.generator(p_neg.coef)
        return p_neg

    def __pos__(self):
        p_pos = self.p %self.poly_modulus
        p_pos = self.generator(p_pos.coef)
        return p_pos

    def __truediv__ (self, scale):
        p_div = self.p / scale
        p_div = self.generator(p_div.coef)
        return p_div

    def __call__(self, x):
        return self.p(x)

    def __repr__(self):
        return self.p.__repr__()

We can see it in use here, we can directly add and multiply polynomials.

In [4]:
N = 4
q = 2**16

polynomial_generator = QPolynomialGenerator(N, q)
a = polynomial_generator([1,0,0,1])
b = polynomial_generator([0,0,0,1])

In [5]:
a

Polynomial([1., 0., 0., 1.], domain=[-1,  1], window=[-1,  1])

Addition works as expected:

In [6]:
a + b

Polynomial([1., 0., 0., 2.], domain=[-1,  1], window=[-1,  1])

The good thing is to see that multiplications are done modulo $X^N + 1$, therefore we have that $a * b = X^3 (X^3 + 1) = X^6 + X^3 = X^4 * X^2 + X^3 = (-1) X^2 + X^3 = X^3 - X^2$:

In [7]:
a * b

Polynomial([ 0.,  0., -1.,  1.], domain=[-1,  1], window=[-1,  1])

Now we will define `PolynomialSampler`, a base class to generate polynomials randomly. This will be inherited by the distributions we will use.

In [8]:
class PolynomialSampler:
    """Base class to sample polynomials."""
    def __init__(self, context):
        self.context = context
        
    def __getattr__(self,k):
        """Context variables are directly linked to the instance."""
        return getattr(self.context, k)
        
    def polynomial(self, coeffs):
        """Create a polynomial using the context."""
        p = self.context.QPolynomial(coeffs)
        return p

In [9]:
class DG(PolynomialSampler):
    def sample(self):
        coeffs = np.random.normal(np.zeros(self.N), self.sigma)
        
        coeffs = np.array(coordinate_wise_random_rounding(coeffs))
        coeffs = mod_q(coeffs, self.q)
        
        p = self.polynomial(coeffs)
        return p

In [ ]:
dg = DG()
dg.s

We can now define the random polynomial samplers needed for key generation:

In [23]:
class UniformPolynomial(PolynomialSampler):
    def sample(self) -> QPolynomial:
            
        coeffs = np.random.choice(self.q, size=self.N)
        coeffs = mod_q(coeffs, self.q)
        
        p = self.polynomial(coeffs)
        return p
    
    def sample_manually(self, q) -> QPolynomial:
        coeffs = np.random.choice(q, size=self.N)
        coeffs = mod_q(coeffs, q)
        
        p = self.polynomial(coeffs)
        return p

In [24]:
class ZO(PolynomialSampler):
    def sample(self):
        coeffs = np.random.choice([0,1,-1], size=self.N, p=[1-self.p, self.p /2, self.p/2])
        
        p = self.polynomial(coeffs)
        return p

In [25]:
class HWT(PolynomialSampler):
    def sample(self):
        coeffs = np.random.choice([-1,1], size=self.N)
        
        slots = np.random.choice(range(self.N), self.N - self.h, replace=False)
        
        coeffs[slots] = 0
            
        p = self.polynomial(coeffs)
        return p

In [16]:
class Context:
    def __init__(self, N, moduli):
        self.N = N
        self.M = N * 2
        
        self.q = np.cumprod(moduli[:-1])[-1]
        self.special_prime = moduli[-1]
        
        self.QPolynomial = QPolynomialGenerator(N, self.q)
        
        self.setup_parameters()
        
        self.hwt = HWT(self)
        self.dg = DG(self)
        self.uniform = UniformPolynomial(self)
        self.zo = ZO(self)
        
    def setup_parameters(self):
        self.h = 64
        self.sigma = 3
        self.p = 0.5

In [17]:
class Keygen:
    def __init__(self, context):
        self.context = context
        
    def generate_secret_key(self):
        s = self.context.hwt.sample()
        return s
    
    def generate_public_key(self, s):
        a = self.context.uniform.sample()
        e = self.context.dg.sample()
        
        b = -(a * s) + e

        pk = (b, a)
        return pk
    
    def generate_relin_key(self, s):
        a = self.context.uniform.sample_manually(self.context.q * self.context.special_prime)
        e = self.context.dg.sample()
        
        b = -(a * s) + e + self.context.special_prime * s * s
        relin_key = (b,a)
        return relin_key

In [18]:
class Encryptor:
    def __init__(self, context, pk):
        self.pk = pk
        self.context = context
        
    def encrypt(self, ptx):
        v = self.context.zo.sample()
        e0 = self.context.dg.sample()
        e1 = self.context.dg.sample()

        v_pk = (self.pk[0] * v, self.pk[1] * v)
        ctx = (v_pk[0] + e0 + ptx, v_pk[1] + e1)
        
        return ctx

In [19]:
class Decryptor:
    def __init__(self, context, s):
        self.s = s
        self.context = context
        
    def decrypt(self, ctx):
        ptx = ctx[0] + self.s * ctx[1]
        return ptx

In [20]:
N = 512
M = N * 2
scale = pow(2,15)

moduli = [scale * pow(2,5), scale, scale * pow(2,5)]

In [22]:
context = Context(N, moduli)
keygen = Keygen(context)

In [23]:
s = keygen.generate_secret_key()
pk = keygen.generate_public_key(s)
relin_key = keygen.generate_relin_key(s)

In [24]:
encoder = CKKSEncoder(context, scale)

In [25]:
encryptor = Encryptor(context, pk)
decryptor = Decryptor(context, s)

In [26]:
z = np.arange(N//2)

In [27]:
ptx = encoder.encode(z)

In [28]:
ctx = encryptor.encrypt(ptx)
ptx = decryptor.decrypt(ctx)
encoder.decode(ptx)

array([-1.57280716e-02-4.46050954e-03j,  1.03442567e+00-3.58267220e-03j,
        2.01265493e+00+1.30881710e-02j,  3.00760827e+00+2.16980451e-02j,
        4.04010763e+00+5.49181320e-02j,  4.94850552e+00-2.65771067e-02j,
        6.00117270e+00-3.68379921e-03j,  6.98699589e+00-2.55515916e-02j,
        7.99722069e+00-2.55954594e-02j,  9.00319157e+00+2.61870886e-02j,
        1.00069622e+01+2.32724669e-02j,  1.10042072e+01-2.11318030e-02j,
        1.19946885e+01-3.89330243e-03j,  1.30295089e+01-4.93250780e-02j,
        1.39443021e+01+3.87786766e-02j,  1.49963635e+01-1.15161159e-02j,
        1.60585743e+01+4.33604859e-02j,  1.70240144e+01-6.20803925e-03j,
        1.80326664e+01-6.83078984e-04j,  1.90120357e+01+2.68526646e-03j,
        1.99860772e+01+1.71128314e-02j,  2.09903604e+01-2.20889133e-02j,
        2.20488862e+01+3.23582278e-02j,  2.29758643e+01+1.34344601e-02j,
        2.40086834e+01+4.16482599e-03j,  2.50168795e+01-2.08137464e-04j,
        2.60191169e+01+3.36725686e-02j,  2.69969655

In [29]:
ct_add = (ctx[0] + ctx[0], ctx[1] + ctx[1])
ptx = decryptor.decrypt(ct_add)
encoder.decode(ptx)

array([-3.14561432e-02-8.92101907e-03j,  2.06885135e+00-7.16534440e-03j,
        4.02530987e+00+2.61763420e-02j,  6.01521654e+00+4.33960901e-02j,
        8.08021526e+00+1.09836264e-01j,  9.89701104e+00-5.31542134e-02j,
        1.20023454e+01-7.36759841e-03j,  1.39739918e+01-5.11031832e-02j,
        1.59944414e+01-5.11909188e-02j,  1.80063831e+01+5.23741771e-02j,
        2.00139245e+01+4.65449338e-02j,  2.20084145e+01-4.22636061e-02j,
        2.39893770e+01-7.78660486e-03j,  2.60590178e+01-9.86501559e-02j,
        2.78886043e+01+7.75573532e-02j,  2.99927269e+01-2.30322317e-02j,
        3.21171487e+01+8.67209717e-02j,  3.40480289e+01-1.24160785e-02j,
        3.60653329e+01-1.36615797e-03j,  3.80240714e+01+5.37053291e-03j,
        3.99721543e+01+3.42256627e-02j,  4.19807207e+01-4.41778267e-02j,
        4.40977724e+01+6.47164556e-02j,  4.59517285e+01+2.68689202e-02j,
        4.80173667e+01+8.32965199e-03j,  5.00337591e+01-4.16274929e-04j,
        5.20382339e+01+6.73451373e-02j,  5.39939310

In [101]:
a = np.ones(encoder.slot_count)
b = np.ones(encoder.slot_count) * 5

In [102]:
ptx1 = encoder.encode(a)
ptx2 = encoder.encode(b)

In [103]:
ctx1 = encryptor.encrypt(ptx1)
ctx2 = encryptor.encrypt(ptx2)

In [104]:
ct_mul = (ctx1[0] * ctx2[0], ctx1[1] * ctx2[0] + ctx1[0] * ctx2[1], ctx1[1] * ctx2[1])

In [105]:
d = (ct_mul[2] * relin_key[0] / context.special_prime, ct_mul[2] * relin_key[1] / context.special_prime)

In [106]:
d = (context.QPolynomial(coordinate_wise_random_rounding(d[0].coef)), 
     context.QPolynomial(coordinate_wise_random_rounding(d[1].coef)))

In [107]:
ct_relin = (ct_mul[0] + d[0], ct_mul[1] + d[1])

In [108]:
ptx = decryptor.decrypt(ct_relin)
encoder.decode(ptx) / scale

array([ 3.67942309e+12+1.44222254e+13j,  1.72190318e+12-6.71147807e+12j,
       -9.63824524e+12+3.09514183e+12j, -1.53583301e+13+2.48433341e+12j,
       -5.84136476e+12-9.41468967e+12j,  5.79015404e+12+7.93774668e+12j,
        4.15217958e+12-7.77578484e+12j,  1.22094194e+12+6.65815750e+12j,
        5.03807402e+12+5.44352568e+12j,  4.33078783e+12-8.32285611e+12j,
       -5.09398186e+12-1.81276414e+13j,  1.31595816e+13-1.33159869e+13j,
        6.44714033e+12+8.43099497e+12j,  3.85575296e+12+1.81740218e+11j,
       -9.73212589e+12+1.48712206e+13j, -1.02839119e+13+4.17555812e+12j,
        2.29673298e+12-5.58790190e+12j,  1.31368059e+13-1.05096267e+13j,
        4.44610683e+12-5.54353324e+12j,  2.69656638e+12+9.04671654e+12j,
        3.15566930e+12-1.48968755e+13j,  6.30210709e+12+8.88434453e+12j,
       -6.03966568e+12-1.39449105e+13j,  2.32434458e+13-3.37212146e+12j,
        1.49180933e+13+2.02971954e+12j,  2.47705488e+13+8.93393803e+12j,
       -2.67876684e+12-1.84447872e+13j, -2.31294214

In [109]:
pt_mul = ct_mul[0] + ct_mul[1] * s + ct_mul[2] * s * s

In [110]:
encoder.decode(pt_mul) / scale

array([ 5.54214881e+02-2.89972637e+02j,  6.81777741e+02+7.48734060e+02j,
        5.26697036e+02-3.75245932e+01j, -1.52396108e+02-7.01592983e+01j,
       -6.70670510e+02-2.20930285e+02j, -3.29489140e+01+2.92085089e+02j,
       -2.26052514e+00-5.07715403e+02j,  2.27298545e+01+1.94358918e+02j,
       -1.52586823e+02+1.39701539e+02j, -3.41207426e+02+1.15570690e+02j,
       -3.32481562e+01+7.05404080e+02j,  2.82900981e+02+5.29701631e+02j,
        3.19832364e+02+3.56942833e+01j, -3.09452562e+02-3.09068446e+01j,
       -1.71144758e+02+1.18537633e+03j,  3.83627614e+02+7.35277388e+01j,
        2.22443710e+02+1.05744502e+02j,  2.56457586e+02+3.61654476e+02j,
        2.46540987e+02+8.32959592e+02j,  4.20249338e+02-7.40552205e+01j,
        1.87535618e+02-3.02408535e+02j, -9.30404983e+02-9.07204549e+01j,
        2.26995044e+02+2.43341447e+02j, -2.06210182e+02-3.22705225e+00j,
        1.80085368e+00+3.78391011e+02j,  6.35774765e+01-6.19547179e+02j,
       -4.34222117e+02-1.88768667e+03j, -1.71133170